## Docker related information

https://github.com/jupyter/docker-stacks

localnotebookpath='/Users/maheshgoud/Documents/Git_Modules/gh/sandbox/'

localdatapath='/Users/maheshgoud/Downloads/data/'

#allsparkpackagenames='org.apache.spark:spark-avro_2.11:2.4.0,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.2'

docker run -it --rm -p 8889:8888 -v $localnotebookpath:/home/jovyan -v $localdatapath:/home/jovyan/data -e JUPYTER_ENABLE_LAB=yes -e SPARK_OPTS='--packages org.apache.spark:spark-avro_2.11:2.4.0,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.2' \
-e PYSPARK_SUBMIT_ARGS='--packages com.databricks:spark-avro_2.10:2.0.1 pyspark-shell' jupyter/all-spark-notebook:7d427e7a4dde start-notebook.sh --NotebookApp.token=''


## Helpful code snippets

- https://www.programcreek.com/scala/org.apache.spark.SparkConf (code snippets)
- https://gist.github.com/eddies/f37d696567f15b33029277ee9084c4a0 (s3 access, packages related)

In [ ]:
import scala.collection.Seq
import sys.process._

// import spark.implicits._
// import org.apache.spark.sql.Row

println("Spark version: "+sc.version)
println("Scala version: "+util.Properties.versionString)
"python --version"!

In [ ]:
// Tinker with functional pattern + daily agg window function

import org.apache.spark.sql.expressions.{Window, WindowSpec}
import org.apache.spark.sql.functions.{col}
val F = org.apache.spark.sql.functions

val df1 = Seq(
      (1, 1, 11, "ord1", "a"),
      (2, 1, 22, "ord2", "a"),
      (5, 5, 13, "ord5", "a"),
      (6, 5, 14, "ord6", "b"),
      (3, 3, 33, "ord3", "b"),
      (4, 3, 44, "ord4", "b")).toDF("DATETIME", "DAY", "VAL", "ORD", "IP")
print(df1.show())
val w = Window.partitionBy(col("IP")).orderBy("DAY").rangeBetween(1, 4)

val df2 = df1.groupBy(col("IP"), col("DAY")).agg(F.sum(col("VAL")).alias("DAILY_VAL"), F.collect_set(col("ORD")).alias("DAILY_ORD")).toDF()
print(df2.show())

// compute collect set over daily collect set aggs
val w2 = Window.orderBy("DAY").rangeBetween(-5, 0)
// print(df2.withColumn("z1", F.flatten(F.collect_set(col("DAILY_ORD")).over(w2))).withColumn("z2", F.size(col("z1"))).show())
print(df2.withColumn("z1", F.size(F.flatten(F.collect_set(col("DAILY_ORD")).over(w2)))).show())

val df3 = df2.withColumn("WINDOWED_VAL", F.sum(col("DAILY_VAL")).over(w))
print(df3.show())

val df4 = df3.withColumn("ORD", F.explode(col("DAILY_ORD"))).drop("DAILY_ORD")
print(df4.show())


In [ ]:
////Test groupby agg exprs
import org.apache.spark.sql.Column

// //ver1
// println(df1.groupBy(col("IP"), col("DAY")).agg(F.sum(col("VAL")).alias("DAILY_VAL"), F.sum(col("DATETIME")).alias("DAILY_DATETIME")).toDF().show())
// val exprs1 = Seq("VAL","DATETIME").map((_ -> "sum")).toMap
// println(exprs1)
// println(df1.groupBy(col("IP"), col("DAY")).agg(exprs1).toDF().show()) 

// println("-"*50)

//ver2
val mapping: Map[String, Column => Column] = Map("MIN" -> F.min, "MAX" -> F.max, "MEAN" -> F.avg, "SUM" -> F.sum, "COUNT" -> F.count)
val operations = Seq("MIN", "MAX", "SUM", "COUNT")
val exprs2 = List("VAL","DATETIME").flatMap(c => operations.map(f => mapping(f)(col(c)).alias(s"DAILY_${f}_${c}")))
println(exprs2)
println(df1.groupBy(col("IP"), col("DAY")).agg(exprs2.head, exprs2.tail: _*).show) 


In [ ]:
//// Tinker with null values based on dataframe col type, append df, foldleft df join, filter type check

// val schema = StructType(Array(
//     StructField("DATETIME", LongType, true),
//     StructField("DAY", LongType, true),
//     StructField("IP", StringType, true),
//     StructField("ORD", StringType, true),
//     StructField("VAL", LongType, true)))
     
// val s = Seq(
//       (1, 1, 11, "ord1", "a"),
//       (2, 1, 22, "ord2", "a"),
//       (5, 5, 13, "ord5", "a"),
//       (6, 5, 14, "ord6", "b"),
//       (3, 3, 33, "ord3", "b"),
//       (4, 3, 44, "ord4", "b")).toDF("DATETIME", "DAY", "VAL", "ORD", "IP")

//// null check
// df4.schema.fields(0).dataType
// df4.schema.fieldIndex("IP")

////foldleft df join
val df1 = Seq(
      (1, 1, 11, "ord1", "a"),
      (2, 1, 22, "ord2", "a"),
      (5, 5, 13, "ord5", "a"),
      (6, 5, 14, "ord6", "b"),
      (3, 3, 33, "ord3", "b"),
      (4, 3, 44, "ord4", "b")).toDF("DATETIME", "DAY", "VAL", "ORD", "IP")

val df10 = Seq(
      (1, 1, 11, "ord1", "a"),
      (2, 1, 22, "ord2", "a"),
      (5, 5, 13, "ord5", "a"),
      (6, 5, 14, "ord6", "b"),
      (3, 3, 33, "ord3", "b"),
      (3, 3, 33, "ord33", "b"),
      (4, 3, 44, "ord4", "b")).toDF("DATETIME", "DAY", "VAL", "ORD", "IP")

val df11 = df1.withColumn("C1", F.lit(1)).withColumn("C2", F.lit(2))
val df12 = df1.withColumn("C3", F.lit(3)).withColumn("C4", F.lit(4))
val df13 = df1.filter($"ORD" =!= "ord1" && $"ORD" =!= "ord2")

val dfs = Seq(df11, df12, df13)
val fdf = dfs.foldLeft(df10)(
  (acc, df) => acc.join(df, Seq("ORD"), "left")
)
print(fdf.show())
// print(fdf.na.fill(1).show())
// fdf.agg(F.sum("C1")).show()
// df10.filter(col("IP").isNotNull && col("IP") =!= 1).show()

////filter type check
// df10.filter(col("IP").isNotNull && col("IP") =!= 2).show() //string
// df10.filter(col("DAY").isNotNull && col("DAY") =!= "11.0").show() //int

In [ ]:
//Declare some dummy dataframe

val df = Seq(
      (1, "foo"),
      (2, "barrio"),
      (3, "gitten"),
      (4, "baa")).toDF("id", "words")

// val someData = Seq(
//   Row(8, "bat"),
//   Row(64, "mouse"),
//   Row(-27, "horse")
// )

// val someSchema = List(
//   StructField("number", IntegerType, true),
//   StructField("word", StringType, true)
// )

// val someDF = spark.createDataFrame(
//   spark.sparkContext.parallelize(someData),
//   StructType(someSchema)
// )

// dictionary Set of words to check 
val d = Set("foo","bar","baa")

println(s"Num of partitions:${df.rdd.getNumPartitions},  All partition sizes:[${df.rdd.glom().map(_.length).collect().mkString(",")}]")

In [ ]:
df.withColumn("isValid", $"words".isInCollection(d)).show()
df.filter($"words".isInCollection(d)).show()

In [ ]:
//Read avro data from s3
// https://github.com/julianpeeters/avro-spark-examples/blob/master/src/main/scala/AvroSparkScala.scala

// import org.apache.avro.generic.GenericRecord
// import org.apache.avro.mapred.AvroKey
// import org.apache.avro.mapreduce.AvroKeyInputFormat
// import org.apache.hadoop.io.NullWritable
// val rdd = sc.newAPIHadoopFile(local_path,
//   classOf[AvroKeyInputFormat[GenericRecord]],
//   classOf[AvroKey[GenericRecord]],
//   classOf[NullWritable]
// )
// rdd.collect()

// s3a://key:secret@
val local_path = "./data/part.avro"

val conf = sc.hadoopConfiguration
conf.set("fs.s3a.access.key", "")
conf.set("fs.s3a.secret.key", "")

val df = spark.read
    .format("com.databricks.spark.avro")
    .load(local_path)
df.collect()